### Compute all of the Dataset Values Given a set of Vectors ###

In [6]:
from HullParameterization import Hull_Parameterization as HP

import numpy as np

import csv
import multiprocessing as mp
from tqdm import tqdm

import ModifiedMichellCw as MWCw
import MaxBox as MB
import Gaussian_Curvature as GC


# Load the Dataset

DS_Path = './'


Vec = np.load('PATH TO DATASET VECTORS')

#Vec = Vec[0:10]

x_labels = np.load('./X_LABELS.npy')

In [7]:
#Multiprocessing
def run_MAP_multiprocessing(func, argument_list, chunksize = None, show_prog = True):
    """Run function in parallel
    Parameters
    ----------
    func:          function
                    Python function to run in parallel.
    argument_list: list [N]
                    List of arguments to be passed to the function in each parallel run.
            
    show_prog:     boolean
                    If true a progress bas will be displayed to show progress. Default: True.
    Returns
    -------
    output:        list [N,]
                    outputs of the function for the given arguments.
    """
    #Reserve 2 threads for other Tasks
    #pool = mp.Pool(processes=mp.cpu_count()-2)
    
    if show_prog:            
        result_list_tqdm = []
        for result in tqdm(pool.map(func=func, iterable=argument_list,chunksize=chunksize), total=len(argument_list),position=0, leave=True):
            result_list_tqdm.append(result)
    else:
        result_list_tqdm = []
        for result in pool.map(func=func, iterable=argument_list,chunksize=chunksize):
            result_list_tqdm.append(result)

    return result_list_tqdm


def run_IMAP_multiprocessing(func, argument_list, chunksize = None, show_prog = True):
    """Run function in parallel
    Parameters
    ----------
    func:          function
                    Python function to run in parallel.
    argument_list: list [N]
                    List of arguments to be passed to the function in each parallel run.
            
    show_prog:     boolean
                    If true a progress bas will be displayed to show progress. Default: True.
    Returns
    -------
    output:        list [N,]
                    outputs of the function for the given arguments.
    """
    #Reserve 2 threads for other Tasks
    #pool = mp.Pool(processes=mp.cpu_count()-2)
    
    if show_prog:            
        result_list_tqdm = []
        for result in tqdm(pool.imap(func=func, iterable=argument_list,chunksize=chunksize), total=len(argument_list),position=0, leave=True):
            result_list_tqdm.append(result)
    else:
        result_list_tqdm = []
        for result in pool.imap(func=func, iterable=argument_list,chunksize=chunksize):
            result_list_tqdm.append(result)

    return result_list_tqdm

In [8]:
# Geometric Measures

z_idx = [12,25,33,38,50,62,67,75,88,100]
        

z_idx_str = np.array([str(np.around(i/100.0, decimals=2)) for i in z_idx])
print(z_idx_str)

def Calc_GeometricProperties(x):
    '''
    This function takes in a Ship Design Vector and calculates the volumetric properties of the hull 
    
    It returns the values for:
    
    Z / L             -> nondimensialized vector for the height at which each value was measured
    Volume / L^3
    Area of Waterplane / L^2
    Longitudinal Centers of Buoyancy/L
    Vertical Center of Buoyancy / L
    Longitudinal Center of Flotation / L
    Ixx / L^4
    Iyy / L^4
    
    where L = LOA of the design vector ( x[0])
    
    This function is written to be paralellized   
    
    '''
    
    hull = HP(x)
       
    Z = hull.Calc_VolumeProperties(NUM_WL = 101, PointsPerWL = 1000)
    
    L = x[0]
    
    z = np.divide(Z[z_idx],L)
    Vol = np.divide(hull.Volumes[z_idx],L**3.0)
    WP = np.divide(hull.Areas_WP [z_idx],L**2.0)
    LCF = np.divide(hull.LCFs[z_idx],L)
    Ixx = np.divide(hull.I_WP[z_idx][:,0],L**4.0)
    Iyy = np.divide(hull.I_WP[z_idx][:,1],L**4.0)
    LCB = np.divide(hull.VolumeCentroids[z_idx][:,0],L)
    VCB = np.divide(hull.VolumeCentroids[z_idx][:,0],L)
    WSA = np.divide(hull.Area_WS[z_idx],L**2.0)
    WL = np.divide(hull.WL_Lengths[z_idx],L)
    
    
    return np.concatenate((z,Vol,WP,LCB,VCB,LCF,Ixx,Iyy,WSA,WL),axis=0)

['0.12' '0.25' '0.33' '0.38' '0.5' '0.62' '0.67' '0.75' '0.88' '1.0']


In [9]:


# Save as input vectors to DSpath

f = open(DS_Path + '/Input_Vectors.csv', 'w')

writer = csv.writer(f)

writer.writerow(x_labels)

for i in range(0,len(Vec)):
    writer.writerow(Vec[i])
    
f.close()    

In [10]:
# Run Multiprocessing to Calculate the Geometric Measures
CHUNKS = 4

print('Threads: ' + str(mp.cpu_count()))

pool = mp.Pool(processes=mp.cpu_count()-2)

output = run_IMAP_multiprocessing(Calc_GeometricProperties, Vec,chunksize=CHUNKS,show_prog=True)

output = np.array(output)

print('Output shape: ' + str(np.shape(output))) # Should be (len(DesVec),len(z_idx *8)) or (Len,100)


Threads: 32


100%|██████████| 41752/41752 [14:26<00:00, 48.19it/s]

Output shape: (41752, 100)


In [11]:
#Save Geometric Measures

# Save all of the data for each calculation

# (z,Vol,WP,LCB,VCB,LCF,Ixx,Iyy)

L = len(z_idx)


savePath = DS_Path + '/GeometricMeasures/'

#save z values
f = open(savePath + 'z.csv', 'w')

writer = csv.writer(f)
writer.writerow(['z/LOA @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,0:L])

f.close()

#save Volume
f = open(savePath + 'Volume.csv', 'w')

writer = csv.writer(f)
writer.writerow(['Volume/LOA^3 @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,L:2*L])

f.close() 
                
#save WaterPlane Area
f = open(savePath + 'Area_WP.csv', 'w')

writer = csv.writer(f)
writer.writerow(['Area_WP/LOA^2 @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,2*L:3*L])

f.close() 

                
#save LCB
f = open(savePath + 'LCB.csv', 'w')

writer = csv.writer(f)
writer.writerow(['LCB/LOA @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,3*L:4*L])

f.close()
                
#save VCB
f = open(savePath + 'VCB.csv', 'w')

writer = csv.writer(f)
writer.writerow(['VCB/LOA @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,4*L:5*L])

f.close() 

#save LCF
f = open(savePath + 'LCF.csv', 'w')

writer = csv.writer(f)
writer.writerow(['LCF/LOA @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,5*L:6*L])

f.close()

#save Ixx
f = open(savePath + 'Ixx.csv', 'w')

writer = csv.writer(f)
writer.writerow(['Ixx/LOA^4 @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,6*L:7*L])

f.close()

#save Iyy
f = open(savePath + 'Iyy.csv', 'w')

writer = csv.writer(f)
writer.writerow(['Iyy/LOA^4 @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,7*L:8*L])

f.close()

#save Wetted Surface Area
f = open(savePath + 'Area_WS.csv', 'w')

writer = csv.writer(f)
writer.writerow(['Area_WS/LOA^2 @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,8*L:9*L])

f.close()

#save WL Length
f = open(savePath + 'WL_Length.csv', 'w')

writer = csv.writer(f)
writer.writerow(['WL/LOA @ T/Dd = ' + i for i in z_idx_str])
for j in range(0,len(output)):
    writer.writerow(output[j,9*L:10*L])

f.close()
                
                
                

In [12]:
# Generate Cw Predictions

z_ref = np.array([0.25,0.33,0.5,0.67]) #This gets the Z values that are 25%,33%,50%, and 67% of Dd
z_idx = np.array([1,2,4,6])
      
Fn = np.array([0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45]) # Froude Numbers

g = 9.81 # Gravity

Rho = 1000 # Density of Water

N = 21 #Number of Sample Angles for Rw calculation

def Calc_Rw(x):
    '''
    This function takes in a Ship Design Vector and calculates the Cw for 4 drafts and 8 speeds for a total of 32 speed conditions for each hull
    
    This function is written to be paralellized   
    
    '''
    Rw = np.zeros((len(z_ref)*len(Fn)))
    
    Hull = HP(x)
    
    for i in range(0,len(z_ref)):
        X,Z,Y,WL   = Hull.gen_PC_for_Cw(Hull.Dd*z_ref[i], NUM_WL=51, PointsPerWL=301)
        
        for j in range(0,len(Fn)):
            U = Fn[j] * np.sqrt(g*WL)
            Rw[i*len(Fn)+j] = MWCw.ModMichell(Y,U,X,Z,Rho,N)

    
    
    return Rw
    
    

In [13]:
#Set Up and Calc Values and save for each design set

DesVec = Vec.astype(np.float64())

print('Design Vector Size: ' + str(np.shape(DesVec)))

# Run Multiprocessing to Calculate the 


print('Threads: ' + str(mp.cpu_count()))

pool = mp.Pool(processes=mp.cpu_count()-2)

output = run_IMAP_multiprocessing(Calc_Rw, DesVec,chunksize=CHUNKS,show_prog=True)
    
'''    
output = []
for j in range(0,1):
    output.append(Calc_Rw(DesVec[j]))
''' 

output = np.array(output)

print('Output shape: ' + str(np.shape(output))) # Should be (len(DesVec),32) or (10000,32)



np.save(savePath + 'Rw_Output.npy', output)



Design Vector Size: (41752, 45)
Threads: 32


  0%|          | 0/41752 [00:00<?, ?it/s]/home/ada/Documents/HullParameterization/ModifiedMichellCw.py:70: RuntimeWarning: overflow encountered in exp
  w0 = (np.exp(Kz)-1.0-Kz)/Kzsq   #Note : np.expm1(x)  = (e^x - 1)
/home/ada/Documents/HullParameterization/ModifiedMichellCw.py:71: RuntimeWarning: overflow encountered in exp
  wn = (np.exp(Kz)-1.0 + np.exp(-Kz)-1.0)/Kzsq
/home/ada/Documents/HullParameterization/ModifiedMichellCw.py:70: RuntimeWarning: overflow encountered in exp
  w0 = (np.exp(Kz)-1.0-Kz)/Kzsq   #Note : np.expm1(x)  = (e^x - 1)
/home/ada/Documents/HullParameterization/ModifiedMichellCw.py:71: RuntimeWarning: overflow encountered in exp
  wn = (np.exp(Kz)-1.0 + np.exp(-Kz)-1.0)/Kzsq
/home/ada/Documents/HullParameterization/ModifiedMichellCw.py:70: RuntimeWarning: overflow encountered in exp
  w0 = (np.exp(Kz)-1.0-Kz)/Kzsq   #Note : np.expm1(x)  = (e^x - 1)
/home/ada/Documents/HullParameterization/ModifiedMichellCw.py:70: RuntimeWarning: overflow encountered in exp
  w0

Output shape: (41752, 32)


In [14]:
'''
Save all of the data for each calculation

INDEXING

Rw[h,i*len(Fn)+j] =  hth hull, i_th draft, speed = Fn[j]
# 
'''


Labels = []

for i in range(0,len(z_ref)):
    for j in range(0,len(Fn)):
        Labels.append('draft=' + str(z_ref[i]) + ' Fn=' + str(Fn[j]))

print(len(Labels))

L = 10.0 # Length of the Test Hulls




WLVec = []
#Load We
with open(savePath + 'WL_Length.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        WLVec.append(row)

#Save as a np.float array
WLVec = np.array(WLVec[1:])
WL = WLVec.astype(np.float64())

print(np.shape(WL))

Cw = np.zeros((len(output),len(output[0])))

print(np.shape(Cw))

for h in range(0,len(output)):
    
    if h % 100 == 0:
        print(h)
    
    for i in range(0,len(z_ref)):
        for j in range(0,len(Fn)):
            U = Fn[j]*np.sqrt(g*WL[h,z_idx[i]]*L)
            Cw[h,i*len(Fn)+j] = output[h,i*len(Fn)+j]/(0.5*Rho*(U**2.0)*L**2.0) 


#save WL Length
f = open(savePath + 'Cw.csv', 'w')

writer = csv.writer(f)
writer.writerow(Labels)
for i in range(0,len(Cw)):
    writer.writerow(Cw[i])

f.close()

32
(41752, 10)
(41752, 32)
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000

In [15]:
# Calculate MaxBox

print('Threads: ' + str(mp.cpu_count()))

pool = mp.Pool(processes=mp.cpu_count()-2)

output = run_IMAP_multiprocessing(MB.Run_BoxOpt, Vec,chunksize=CHUNKS,show_prog=True)

Boxes = np.array(output)

print('PC Con Violation Count: ' + str(sum(1 for a in Boxes[:,5] if a)))

Threads: 32


  0%|          | 0/41752 [00:00<?, ?it/s]


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation


Compiled modules for significant speedup can not be used!To disable this warning:

https://pymoo.org/installation.html#installation
from pymoo.config import Config

Compiled modules for significant speedup can not be used!To disable this warning:
from pymoo.config impo

100%|██████████| 41752/41752 [13:30:40<00:00,  1.16s/it]  

PC Con Violation Count: 0


In [16]:
f = open(savePath + 'MaxBox.csv', 'w')
labels = ['x/LOA of fwd end of Box' ,'length/LOA of Box','height/LOA of Box','width/LOA of Box','Volume/LOA^3 of Box']
writer = csv.writer(f)

writer.writerow(labels)
for j in range(0,len(Boxes)):
    writer.writerow(Boxes[j,0:-1])

f.close()

In [17]:
#Calculate Gaussian Curvature


print('Threads: ' + str(mp.cpu_count()))

pool = mp.Pool(processes=mp.cpu_count()-2)

output = run_IMAP_multiprocessing(GC.GaussianCurvature, Vec,chunksize=CHUNKS,show_prog=True)

GK = np.array(output)


Threads: 32


100%|██████████| 41752/41752 [1:11:36<00:00,  9.72it/s]


In [18]:
f = open(savePath + 'GaussianCurvature.csv', 'w')

writer = csv.writer(f)
writer.writerow(['Gaussian Curvature * LOA^2'])
for j in range(0,len(GK)):
    writer.writerow([GK[j]])

f.close()

In [19]:
#Generate STL

for i in tqdm(range(0,1000)):


    hull = HP(Vec[i])

        
    #PC = hull.gen_pointCloud(NUM_WL = 50, PointsPerWL = 500)

    strpath =  DS_Path + '/stl/Hull_mesh_' + str(i) # + '.stl'

    mesh = hull.gen_stl(NUM_WL= 100, PointsPerWL = 800,  namepath = strpath)

100%|██████████| 1000/1000 [05:15<00:00,  3.17it/s]
